# how to build hakken 


> i will explain how i build the hakken: coding agent. we will cover some fundamental on how you can build agent and then will build some of the features of hakken not all tho but some.

## outline 

- we wil start from simple llm tool call 
- then we will build react agent 
- then we will add some features


> # part-1 : llm tool call 

In [4]:
## imports 
import json, requests
from openai import OpenAI
from tavily import TavilyClient # for web search 
import os 
from dotenv import load_dotenv
load_dotenv()

True

- you can get api for llm (free): [link](https://openrouter.ai/z-ai/glm-4.5-air:free)
- and for our web search tool (free): [link](https://www.tavily.com/)


> ### model : llm 

In [43]:
## lets define our llm 

#get the keys from env 
openrouter_key = os.getenv("OPENROUTER_API_KEY")

openai_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_key,
)



> ### tool description 

In [66]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
from typing import Literal

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

tools = [
    {
        "type": "function",
        "function": {
            "name": "internet_search",
            "description": "Search the internet for current information, news, and research. Can search across general topics, news, or finance-specific content. Useful for finding up-to-date information, research developments, tech news, and current events.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query string to find relevant information"
                    },
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of search results to return (default: 5)",
                        "default": 5
                    },
                    "topic": {
                        "type": "string",
                        "enum": ["general", "news", "finance"],
                        "description": "The topic category to focus the search on",
                        "default": "general"
                    },
                    "include_raw_content": {
                        "type": "boolean",
                        "description": "Whether to include raw content from web pages in the results",
                        "default": False
                    }
                },
                "required": ["query"]
            }
        }
    }
]

TOOL_MAPPING = {
    "internet_search": internet_search
}

In [67]:
task = "can you search new ai agent trends in 2025 ?"

messages = [
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": task, ## our query 
  }
]

> ###  lets run without tool call; just simple llm call 

In [68]:
model_name = os.getenv("OPENAI_MODEL")

request_1 = {
    "model":model_name,
    "tools": tools,
    "messages": messages
}
completion_1 = openai_client.chat.completions.create(**request_1)
response_1 = completion_1.choices[0].message 


In [72]:
print(response_1)

ChatCompletionMessage(content="\nI'll search for the latest AI agent trends in 2025 to keep you updated on this evolving technology.\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_YdopfCm6Sp6LHYhTHLkcSA', function=Function(arguments='{"query": "AI agent trends 2025 new developments", "topic": "general", "max_results": 5}', name='internet_search'), type='function', index=0)], reasoning='\nThe user is asking me to search for new AI agent trends in 2025. This is a current topic request, so I should use the internet_search function to find up-to-date information about AI agent trends for 2025.\n\nI need to:\n1. Use the internet_search function\n2. Set the query to something like "AI agent trends 2025" or "new AI agent developments 2025"\n3. Use the "general" topic since this is about technology trends\n4. I don\'t need to include raw content unless specified, so I\'ll leave that as default fal

In [73]:
tool_call.function.name

'internet_search'

In [75]:
tool_call.function.arguments

'{"query": "AI agent trends 2025 new developments", "topic": "general", "max_results": 5}'

> ### okay its working 

In [ ]:
# append the response to the messages array so the LLM has the full context
messages.append(response_1) ## adding to history 
if response_1.tool_calls:
    for tool_call in response_1.tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        # Check if the tool exists in our mapping
        if tool_name in TOOL_MAPPING:
            tool_response = TOOL_MAPPING[tool_name](**tool_args) # this where we call the function with proper arguments 

            # add tool response to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_response, indent=2),
            })

print(messages) ## full history from steps 1–2 (system + user + assistant/tool messages).


[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'can you search new ai agent trends in 2025 ?'}, ChatCompletionMessage(content="\nI'll search for the latest AI agent trends in 2025 to keep you updated on this evolving technology.\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_YdopfCm6Sp6LHYhTHLkcSA', function=Function(arguments='{"query": "AI agent trends 2025 new developments", "topic": "general", "max_results": 5}', name='internet_search'), type='function', index=0)], reasoning='\nThe user is asking me to search for new AI agent trends in 2025. This is a current topic request, so I should use the internet_search function to find up-to-date information about AI agent trends for 2025.\n\nI need to:\n1. Use the internet_search function\n2. Set the query to something like "AI agent trends 2025" or "new AI agent developments 2025"\n3. Use the "gener

In [ ]:
request_2 = {
    "model": model_name,
    "tools": tools,
    "messages": messages ## we send this to model to get clear response 
}
completion_2 = openai_client.chat.completions.create(**request_2)
final_response = completion_2.choices[0].message
print(final_response.content)




Based on my search, here are the key AI agent trends for 2025:

## Major AI Agent Trends in 2025

### 1. **Enterprise Adoption and ROI Focus**
- Companies are seeing **up to 50% efficiency improvements** in customer service, sales, and HR operations
- ROI is the primary driver for enterprise adoption
- Major cloud providers like AWS are doubling down on AI agent investments, creating dedicated business units

### 2. **Enhanced Autonomy and Capability**
- AI agents are becoming **fully autonomous** - able to scope out projects and complete them with necessary tools without human help
- New generation of agents can **handle tasks on behalf of users** rather than just assisting
- Agents are being integrated into workflow automation systems

### 3. **Democratization of Agent Creation**
- **No-code platforms** like Microsoft's Copilot Studio allow anyone to build AI agents
- Developers have access to sophisticated tools like Azure AI Foundry for creating more complex agents
- Enhanced serv

---
>
> ## we can break above stuff into three step process
>
---
### **first step**
call to model : user's message (and conversation history) to the model. this is the "reasoning" phase where the model decides if tools are needed.

**input**
- `messages`: system prompt + user query

- `tools` : for example intern_search 

- `model_name` : gpt-5 

**ouput**

- `content` : which include the resoning of model 

- `tool_calls`: list of tool requests (if any), each with `name`

**code** : 
```python 
completion_1 = openai_client.chat.completions.create(**request_1)
response_1 = completion_1.choices[0].message
messages.append(response_1)  # Add to history
```
---

### **second step**

`what it does` : if tools exist then parse it and execute the matching function (e.g., internet_search via tavily), and add the results back to the conversation history as "tool" messages

**input**

- `response_1.tool_calls` : list from Step 1 (e.g., one tool call with args like {"query": "AI agent trends 2025"}).

- `TOOL_MAPPING` : dict mapping tool names to functions (e.g., "internet_search": internet_search).


**ouput**

here we get the function name of all tools and arguments of it and then execute the tools and append the all tool response to messeges.


- `tool_response`: raw result from the tool (e.g., tavily search returns a dict with results list of URLs, titles, snippets).

- `Updated messages`: appends a new dict for each tool: {"role": "tool", "tool_call_id": "call_abc123", "content": json.dumps(`tool_response`)}.

**code** 

```python 
if response_1.tool_calls: # seeing if model need tool call to execute query of user is yes then continue otherwise skip 
    for tool_call in response_1.tool_calls:
        tool_name = tool_call.function.name ## getting name of tool eg internet_search 
        tool_args = json.loads(tool_call.function.arguments) ## getting arugement of it query and other 
        if tool_name in TOOL_MAPPING:
            tool_response = TOOL_MAPPING[tool_name](**tool_args) ## here the tool call is happeinig 
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_response, indent=2),
            })
```

----

###  **third step** 

here we send all the history of `step-1+tool_response` to model to get clean ouput 


**Input**:
- `updated messages`: full history from steps 1–2 (system + user + assistant/tool messages).

- `tools`: same tool list (in case more calls are needed, though often not).

**ouput** 

- `final_response` : clean

---
---

> # part-2: react agent 

ReAct agent is made of this four components :

1. thought: internal reasoning about what to do next
2. action: a specific action to take (like searching, calculating, or calling an API)
3. observation: the result of that action
4. repeat until the task is complete

> to get this working we need terminal acess so all the code will be in tutorials/[react_agent](tutorials/react_agent) 